In [1]:
import spkit as sp
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline  
import mne
from copy import deepcopy
from mne.preprocessing import compute_proj_ecg
from mne_connectivity import envelope_correlation
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import random
from mne.preprocessing import ICA, corrmap, create_ecg_epochs, create_eog_epochs
from mne.preprocessing import annotate_amplitude
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from scipy.signal import welch, coherence
from scipy.stats import entropy, skew, kurtosis
import networkx as nx
import time
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
#import tsfresh
#from tsfresh import extract_features
#from tsfresh.feature_extraction import ComprehensiveFCParameters
#from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
import pyrqa
import mne
from mne.decoding import CSP
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import pywt
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.nn import GCNConv
from torchvision.models import mobilenet_v2
from torchvision import transforms
import cv2
from ordpy import renyi_entropy
import joblib

c:\Users\mhfar\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
import logging
logging.getLogger('mne').setLevel(logging.WARNING)

In [3]:
class FPG_Dataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.filepaths, self.labels = self._load_filepaths_and_labels()

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, index):
        filepath = self.filepaths[index]
        label = self.labels[index]

        eeg_data = self._load_eeg(filepath)

        if self.transform is not None:
            eeg_data = self.transform(eeg_data)

        return torch.tensor(eeg_data), torch.tensor(label)

    def _load_filepaths_and_labels(self):
        filepaths = []
        labels = []

        classes = sorted(os.listdir(self.root_dir))
        for class_index, class_name in enumerate(classes):
            class_dir = os.path.join(self.root_dir, class_name)
            if os.path.isdir(class_dir):
                filenames = os.listdir(class_dir)
                for filename in filenames:
                    filepaths.append(os.path.join(class_dir, filename))
                    labels.append(class_index)

        return filepaths, labels

    def _load_eeg(self, filepath):
        data = mne.read_epochs(filepath, preload=False).get_data(picks='eeg');
        normals = []
        scaler = StandardScaler()
        for idx in range(len(data)):
            normals.append(scaler.fit_transform(data[idx]))

        return np.array(normals)

In [4]:
def custom_collate_fn(batch):
    # Concatenate data samples along the first dimension (window_count)
    # Assumes each sample is a tuple (uid, data_sample)
    uids, data_samples = zip(*batch)
    concatenated_data = torch.stack(data_samples, dim=0)
    return uids, concatenated_data

In [5]:
def PLV(sample, verbose=False):
        
        EEG_data = sample[0]
        threshold = 0.3
        G = nx.Graph()

        # Add nodes (brain regions)
        G.add_nodes_from(range(21))  # Assuming 21 brain regions

        # Add edges (based on functional connectivity)
        for i in range(21):
            for j in range(i + 1, 21):
                # Calculate functional connectivity strength between nodes i and j
                # (e.g., using correlation coefficients from EEG signals)

                #phase locking value
                connectivity_strength = np.abs(np.mean(np.exp(1j * np.angle(EEG_data[i] * np.conj(EEG_data[j])))))  # Replace with actual method

                if connectivity_strength > threshold:
                    G.add_edge(i, j, weight=connectivity_strength)

        # Calculate graph metrics
        local_efficiency = nx.local_efficiency(G)
        global_efficiency = nx.global_efficiency(G)

        if (verbose):
                
            print(f"Local efficiency: {local_efficiency}")
            print(f"Global efficiency: {global_efficiency}")

        return local_efficiency, global_efficiency

In [6]:
def RQA(data_points, dimension):
    time_series = TimeSeries(data_points,
                         embedding_dimension = int(dimension),
                         time_delay =2)
    settings = Settings(time_series,
                    analysis_type=Classic,
                    neighbourhood=FixedRadius(0.1),
                    similarity_measure=EuclideanMetric,
                    theiler_corrector=1)
    computation = RQAComputation.create(settings,
                                    verbose=False)
    result = computation.run()
    rqa_features = [
        result.recurrence_rate,
        result.determinism,
        result.divergence,
        result.entropy_diagonal_lines,
        result.laminarity,
        result.trapping_time,
        result.entropy_vertical_lines,
        result.entropy_white_vertical_lines,
    ]
    
    return np.array(rqa_features)

In [7]:
def extract_wavelet_features(signal, wavelet_name, level):
    coeffs = pywt.wavedec(signal, wavelet_name, level=level)
    features = []
    for coeff in coeffs:
        features.extend([np.mean(coeff), np.std(coeff)])
    return features

In [8]:
def shannon_entropy(signal):
    # Compute probability distribution
    unique_values, counts = np.unique(signal, return_counts=True)
    probabilities = counts / len(signal)

    # Calculate Shannon entropy
    entropy = -np.sum(probabilities * np.log2(probabilities))

    return entropy

In [9]:
def PLV(eeg_data):
    PLVs = []
    for channel in range(eeg_data.shape[0]):
        if(channel!=20):
            #print(channel)
            plv = np.abs(np.mean(np.exp(1j * np.angle(eeg_data[channel] * np.conj(eeg_data[channel + 1])))))
            PLVs.append(plv)
        
    PLVs.append(np.abs(np.mean(np.exp(1j * np.angle(eeg_data[20] * np.conj(eeg_data[0]))))))
    return np.array(PLVs).reshape(21, -1)


In [10]:
def coherence_features(eeg_data):
    vals = []
    for channel in range(eeg_data.shape[0]):
        if(channel!=20):
            #print(channel)
            f, Cxy = coherence(eeg_data[channel], eeg_data[channel + 1], fs=300, nperseg=300)
            vals.append(np.mean(Cxy))
        
    f, Cxy = coherence(eeg_data[20], eeg_data[0], fs=300, nperseg=300)
    vals.append(np.mean(Cxy))
    return np.array(vals).reshape(21, -1)


In [11]:
def extract_eeg_features(eeg_data, wavelet_type = 'db4', wavelet_level = 4, include_rqa = False):
    eeg_data = np.array(eeg_data)
    num_channels = eeg_data.shape[0]

    # Initialize arrays to store features
    mean_values = np.zeros(num_channels)
    std_values = np.zeros(num_channels)
    min_values = np.zeros(num_channels)
    max_values = np.zeros(num_channels)
    skewness_values = np.zeros(num_channels)
    kurtosis_values = np.zeros(num_channels)
    p2p_values = np.zeros(num_channels)
    p2rms_values = np.zeros(num_channels)
    rss_values = np.zeros(num_channels)
    amplitude = np.zeros(num_channels)
    rms = np.zeros(num_channels)
    zero_crossing_rate = np.zeros(num_channels)
    delta_power = np.zeros(num_channels)
    theta_power = np.zeros(num_channels)
    alpha_power = np.zeros(num_channels)
    beta_power = np.zeros(num_channels)
    gamma_power = np.zeros(num_channels)
    spectral_entropy = np.zeros(num_channels)
    shannon_ent = np.zeros(num_channels)
    renyi_ent = np.zeros(num_channels)
    rqa_matrix = []

    db_wavelet_features = []
    for channel in eeg_data:
        channel_features = extract_wavelet_features(channel, 'sym5', 4)
        db_wavelet_features.append(channel_features)

    sym_wavelet_features = []
    for channel in eeg_data:
        channel_features = extract_wavelet_features(channel, 'db4', 4)
        sym_wavelet_features.append(channel_features)



    for channel in range(num_channels):
        # Extract data for the current channel
        channel_data = eeg_data[channel]

        # Compute mean, STD, min, max
        mean_values[channel] = np.mean(channel_data)
        std_values[channel] = np.std(channel_data)
        min_values[channel] = np.min(channel_data)
        max_values[channel] = np.max(channel_data)

        # Compute skewness and kurtosis
        skewness_values[channel] = skew(channel_data)
        kurtosis_values[channel] = kurtosis(channel_data)

        # Compute peak-to-peak (P2P)
        p2p_values[channel] = max_values[channel] - min_values[channel]

        # Compute peak-to-root sum square (P2RMS)
        p2rms_values[channel] = np.sqrt(np.sum(channel_data**2))

        # Compute root sum square (RSS)
        rss_values[channel] = np.sqrt(np.sum(channel_data**2))

        # Compute amplitude (peak-to-peak divided by 2)
        amplitude[channel] = p2p_values[channel] / 2

        # Compute root mean square (RMS)
        rms[channel] = np.sqrt(np.mean(channel_data**2))

        # Compute zero-crossing rate
        zero_crossing_rate[channel] = np.mean(np.diff(np.sign(channel_data)) != 0)

        # Compute power spectral density using Welch method
        f, psd = welch(channel_data, fs=1000, nperseg=256)
        delta_power[channel] = np.sum(psd[(f >= 1) & (f <= 4)])
        theta_power[channel] = np.sum(psd[(f >= 4) & (f <= 8)])
        alpha_power[channel] = np.sum(psd[(f >= 8) & (f <= 14)])
        beta_power[channel] = np.sum(psd[(f >= 14) & (f <= 30)])
        gamma_power[channel] = np.sum(psd[f > 30])
        if(include_rqa):
            rqa_matrix.append(RQA(channel_data, dimension=4))

        # Compute spectral entropy
        spectral_entropy[channel] = -np.sum(psd * np.log2(psd))
        #shannon_ent[channel] = shannon_entropy(channel_data)
        #renyi_ent[channel] = renyi_entropy(channel_data, 3)


    # Organize features into a dictionary
    features = np.array([mean_values, std_values, min_values, max_values, skewness_values, kurtosis_values,
                         p2p_values, p2rms_values, rss_values,amplitude, rms, zero_crossing_rate,delta_power,
                         theta_power, alpha_power, beta_power, gamma_power]).T
    if(include_rqa):
        features = np.append(features, np.array(rqa_matrix), axis=1)

    features = np.append(features, db_wavelet_features, axis=1)
    #features = np.append(features, sym_wavelet_features, axis=1)

    #features = np.append(features, PLV(eeg_data), axis = 1)
    #features = np.append(features, coherence_features(eeg_data), axis= 1)
    
    return features

#extracted_features = extract_eeg_features(sample[0])
#extracted_features.shape

In [12]:
def Load_Data(path, wavelet_type, wavelet_level, include_rqa, verbose):
    dataset = FPG_Dataset(root_dir=path)

    batch_size = 1  # Set your desired batch size
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=custom_collate_fn)

    label = []
    feature_space = []
    count = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        for patient in inputs:
            
            for epoch in patient:
                feature_space.append(extract_eeg_features(epoch, wavelet_type, wavelet_level, include_rqa))
                label.append(targets)
            
            if(verbose):
                print("Patient processed")
                print(count)
                count = count + 1
    
    feature_space = np.squeeze(feature_space)
    labels = np.array([tensor.numpy() for tensor in label])
    flat_features = feature_space.reshape(feature_space.shape[0], -1)

 
    return feature_space, labels.reshape(-1)

In [13]:
Dataset = FPG_Dataset(root_dir=r"C:\Users\mhfar\OneDrive\Desktop\20second_MNE_3CLASS")

In [14]:
X_train, y_train = Load_Data(r"C:\Users\mhfar\OneDrive\Desktop\20second_MNE_3CLASS", include_rqa = False, wavelet_type = 'db4', wavelet_level = 9, verbose = True)

Patient processed
0
Patient processed
1
Patient processed
2
Patient processed
3
Patient processed
4
Patient processed
5
Patient processed
6
Patient processed
7
Patient processed
8
Patient processed
9
Patient processed
10
Patient processed
11
Patient processed
12
Patient processed
13
Patient processed
14
Patient processed
15
Patient processed
16
Patient processed
17
Patient processed
18
Patient processed
19
Patient processed
20
Patient processed
21
Patient processed
22
Patient processed
23
Patient processed
24
Patient processed
25
Patient processed
26
Patient processed
27
Patient processed
28
Patient processed
29
Patient processed
30
Patient processed
31
Patient processed
32
Patient processed
33
Patient processed
34
Patient processed
35
Patient processed
36
Patient processed
37
Patient processed
38
Patient processed
39
Patient processed
40
Patient processed
41
Patient processed
42
Patient processed
43
Patient processed
44
Patient processed
45
Patient processed
46
Patient processed
47
Pa

In [15]:
X_test, y_test = Load_Data(r"C:\Users\mhfar\OneDrive\Desktop\20second_MNE_3CLASS_TEST", include_rqa = False, wavelet_type = 'db4', wavelet_level = 9, verbose = True)

Patient processed
0
Patient processed
1
Patient processed
2
Patient processed
3
Patient processed
4
Patient processed
5
Patient processed
6
Patient processed
7
Patient processed
8


In [16]:
eeg_data = Dataset.__getitem__(0)[0][20]

In [17]:
"""
wavelet = 'sym5'
level = 9

wavelet_features = []
for channel in eeg_data:
    channel_features = extract_wavelet_features(channel, wavelet, level)
    wavelet_features.append(channel_features)

wavelet_features = np.array(wavelet_features)
np.shape(wavelet_features)
"""


"\nwavelet = 'sym5'\nlevel = 9\n\nwavelet_features = []\nfor channel in eeg_data:\n    channel_features = extract_wavelet_features(channel, wavelet, level)\n    wavelet_features.append(channel_features)\n\nwavelet_features = np.array(wavelet_features)\nnp.shape(wavelet_features)\n"

In [18]:
csp = CSP(n_components=21, reg=None, log=True, norm_trace=False)
X_train_combined = np.hstack((X_train.reshape(X_train.shape[0], -1), csp.fit_transform(X_train, y_train)))
X_test_combined = np.hstack((X_test.reshape(X_test.shape[0], -1), csp.fit_transform(X_test, y_test)))

In [19]:
X_train_combined.shape

(38724, 588)

In [20]:

start = time.time()
rf_classifier = RandomForestClassifier(n_estimators=80, random_state=42)
knn_classifier = KNeighborsClassifier(n_neighbors=100)
svm_classifier = SVC()
ovr_classifier = OneVsRestClassifier(rf_classifier)
ovr_classifier.fit(X_train_combined, y_train)

OneVsRestClassifier(estimator=RandomForestClassifier(n_estimators=80,
                                                     random_state=42))

In [21]:
y_pred = ovr_classifier.predict(X_test_combined)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(f"training time: {time.time() - start}")

Accuracy: 0.70
training time: 295.1083550453186


In [22]:
joblib.dump(ovr_classifier, "randomforest.joblib")

['randomforest.joblib']

In [21]:
class FeatureGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(FeatureGNN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, num_classes)

    def forward(self, x, edge_index):
        # x: Your feature vector (shape: [num_samples, input_dim])
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Example usage
input_dim = 819  # Dimension of your feature vector
hidden_dim = 400  # Hidden layer dimension
num_classes = 3  # Number of classes (e.g., 0, 1, 2)

# Initialize the GNN model
model = FeatureGNN(input_dim, hidden_dim, num_classes)
edge_index = torch.randint(0, 819, (2, 200))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [22]:
X_train_tensor = torch.tensor(X_train_combined, dtype=torch.float32)
y_train_tensor = torch.nn.functional.one_hot(torch.tensor(y_train, dtype=torch.long).unsqueeze(1), 3).squeeze()
y_train_tensor = torch.tensor(y_train_tensor, dtype=torch.float32)

X_test_tensor = torch.tensor(X_test_combined, dtype=torch.float32)
y_test_tensor = torch.nn.functional.one_hot(torch.tensor(y_test, dtype=torch.long).unsqueeze(1), 3).squeeze()
y_test_tensor = torch.tensor(y_test_tensor, dtype=torch.float32)

C:\Users\mhfar\AppData\Local\Temp\ipykernel_21256\1728568494.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
C:\Users\mhfar\AppData\Local\Temp\ipykernel_21256\1728568494.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


In [ ]:
for epoch in range(100):
    optimizer.zero_grad()
    outputs = model(X_train_tensor, edge_index)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    #print(f"Epoch [{epoch+1}/100], Loss: {loss.item():.4f}")

In [ ]:
predictions = torch.argmax(model(X_test_tensor, edge_index), axis = 1)

In [ ]:

accuracy = accuracy_score(torch.argmax(y_test_tensor, axis = 1), predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.54
